<a href="https://colab.research.google.com/github/BDH-teacher/Deep_Learning_Audit_code/blob/main/Weight_Initialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
np.random.seed(0)

# Weight Initialization
class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.W1 = np.random.rand(input_size, hidden_size) * 0.01  # Input to hidden weights
        self.b1 = np.zeros((1, hidden_size))  # Hidden layer biases
        self.W2 = np.random.rand(hidden_size, output_size) * 0.01  # Hidden to output weights
        self.b2 = np.zeros((1, output_size))  # Output layer biases

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return z * (1 - z)


In [ ]:
# Up until now...
def preprocess_text(text: str):
    # 아주 간단한 전처리: 소문자 + 알파벳/공백만 남기기
    text = text.lower()
    text = ''.join(ch if (ch.isalnum() or ch.isspace()) else ' ' for ch in text)
    tokens = [t for t in text.split() if t]
    return tokens

def create_vocabulary(tokenized_sentences):
    # <pad>를 0으로 두면, 뒤에서 padding 처리/마스킹이 편함
    vocab = ["<pad>", "<unk>"]
    words = sorted({w for sent in tokenized_sentences for w in sent})
    vocab += words
    word_to_index = {w: i for i, w in enumerate(vocab)}
    return vocab, word_to_index

def encode_text(tokenized_sentences, word_to_index):
    max_len = max(len(s) for s in tokenized_sentences)
    X = []
    for sent in tokenized_sentences:
        ids = [word_to_index.get(w, word_to_index["<unk>"]) for w in sent]
        # pad
        ids = ids + [word_to_index["<pad>"]] * (max_len - len(ids))
        X.append(np.array(ids, dtype=np.int64))
    return X

# Sample dataset
sentences = [
    "I love programming",
    "Python is a great language",
    "I enjoy learning new technologies",
    "I hate bugs in code",
    "Debugging is fun",
    "I dislike syntax errors",
]

# Corresponding labels (1 for positive sentiment, 0 for negative)
labels = np.array([[1], [1], [1], [0], [1], [0]])

# Preprocess the text
sentences = [preprocess_text(sentence) for sentence in sentences]

# Create vocabulary and mappings
vocabulary, word_to_index = create_vocabulary(sentences)

# Encode sentences into input IDs and convert into array of input_ids
X = encode_text(sentences, word_to_index)
X = np.array([np.array(xi) for xi in X])

print("X.shape:", X.shape, "| labels.shape:", labels.shape)
print("vocab_size:", len(vocabulary), "| example vocab:", vocabulary[:10])

X.shape: (6, 5) | labels.shape: (6, 1)
vocab_size: 23 | example vocab: ['<pad>', '<unk>', 'a', 'bugs', 'code', 'debugging', 'dislike', 'enjoy', 'errors', 'fun']


In [ ]:
# Initialize Embedding Weight / from scratch

class SimpleNN:
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        # Initialize weights and biases
        self.embedding_weights = np.random.rand(vocab_size, embedding_dim) * 0.01  # Embedding weights
        self.W1 = np.random.rand(embedding_dim, hidden_size) * 0.01  # Hidden layer weights
        self.b1 = np.zeros((1, hidden_size))  # Hidden layer biases
        self.W2 = np.random.rand(hidden_size, output_size) * 0.01  # Output layer weights
        self.b2 = np.zeros((1, output_size))  # Output layer biases

        # 패딩 id는 <pad>=0으로 가정 (위 vocab 생성이 그렇게 만들어둠)
        self.pad_id = 0

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return z * (1 - z)

    def forward(self, X):
        # Forward pass
        # ...
        # return prediction

        # X: (batch, seq_len) token ids
        emb = self.embedding_weights[X]  # (batch, seq_len, embedding_dim)

        mask = (X != self.pad_id).astype(np.float32)  # (batch, seq_len)
        counts = mask.sum(axis=1, keepdims=True)      # (batch, 1)
        counts[counts == 0] = 1.0

        # masked mean pooling
        x_emb = (emb * mask[..., None]).sum(axis=1) / counts  # (batch, embedding_dim)

        z1 = x_emb.dot(self.W1) + self.b1
        h1 = self.sigmoid(z1)

        z2 = h1.dot(self.W2) + self.b2
        prediction = self.sigmoid(z2)  # (batch, 1) 확률

        # backward에서 쓰려고 캐시
        self._cache = {
            "X": X,
            "mask": mask,
            "counts": counts,
            "x_emb": x_emb,
            "h1": h1,
            "prediction": prediction,
        }
        return prediction

    def compute_loss(self, y, output):
        # Compute binary cross-entropy loss
        m = y.shape[0]  # Number of samples
        return -np.sum(y * np.log(output + 1e-15) + (1 - y) * np.log(1 - output + 1e-15)) / m

    def backward(self, X, y, output, learning_rate=0.01):
        #Back Propagation (1. Calculate gradient 2. Update weights and bias using gradient descent)
        # ...

        m = y.shape[0]
        cache = self._cache
        x_emb = cache["x_emb"]      # (m, embedding_dim)
        h1 = cache["h1"]            # (m, hidden)
        mask = cache["mask"]        # (m, seq_len)
        counts = cache["counts"]    # (m, 1)

        # BCE + sigmoid output이면 dL/dz2 = (pred - y)/m
        dz2 = (output - y) / m                 # (m, 1)
        dW2 = h1.T.dot(dz2)                    # (hidden, 1)
        db2 = dz2.sum(axis=0, keepdims=True)   # (1, 1)

        dh1 = dz2.dot(self.W2.T)               # (m, hidden)
        dz1 = dh1 * self.sigmoid_derivative(h1)  # (m, hidden)

        dW1 = x_emb.T.dot(dz1)                 # (emb_dim, hidden)
        db1 = dz1.sum(axis=0, keepdims=True)   # (1, hidden)

        dx_emb = dz1.dot(self.W1.T)            # (m, emb_dim)

        # embedding gradient: mean pooling을 각 토큰에 분배
        grad_embedding = np.zeros_like(self.embedding_weights)
        seq_len = X.shape[1]
        for i in range(m):
            denom = counts[i, 0]
            for j in range(seq_len):
                tid = X[i, j]
                if tid == self.pad_id:
                    continue
                grad_embedding[tid] += dx_emb[i] / denom

        # gradient descent update
        self.embedding_weights -= learning_rate * grad_embedding
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

In [ ]:
# Up until now... (SimpleNN 인스턴스 생성)

# Initialize the neural network
vocab_size = len(vocabulary)  # Number of unique words
embedding_dim = 5  # Embedding dimension
hidden_size = 5  # Number of neurons in the hidden layer
output_size = 1  # One output (binary classification)

nn = SimpleNN(vocab_size, embedding_dim, hidden_size, output_size)


# What's next? Write Train & Test APIs!
def train(X, y, epochs=1000, learning_rate=0.01):
    for epoch in range(epochs):
        output = nn.forward(X)  # Forward pass
        loss = nn.compute_loss(y, output)  # Compute loss
        nn.backward(X, y, output, learning_rate)  # Backward pass

        if epoch % 100 == 0:  # Print loss every 100 epochs
            print(f'Epoch {epoch}, Loss: {loss}')

def predict(X):
    output = nn.forward(X)
    return (output > 0.5).astype(int)  # Binary classification

train(X,labels)
predictions = predict(X)
print(predictions)
print(labels)

Epoch 0, Loss: 0.6903513296925704
Epoch 100, Loss: 0.6545826684507242
Epoch 200, Loss: 0.642744689530779
Epoch 300, Loss: 0.6387065442840889
Epoch 400, Loss: 0.637295732687321
Epoch 500, Loss: 0.6367948569386922
Epoch 600, Loss: 0.6366151983723816
Epoch 700, Loss: 0.63655033053068
Epoch 800, Loss: 0.6365267987304426
Epoch 900, Loss: 0.636518221875147
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 [0]
 [1]
 [0]]


In [ ]:
# PyTorch RNN + DataLoader + Train/Test API

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

torch.manual_seed(0)
vocab_to_int = word_to_index


# Code Implementation: Define a RNN using Pytorch
# Define RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        # Define Embedding
        self.embedding = nn.Embedding(len(vocab_to_int), input_size, padding_idx=vocab_to_int['<pad>'])

        # Define the RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)

        # Define the fully connected layer to produce outputs
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedding = self.embedding(x)

        # Forward propagate the RNN
        out, hidden = self.rnn(embedding)

        # Take the output from the last time step
        out = self.fc(out[:, -1, :])
        return out

# Dataset / split
class TextDataset(Dataset):
    def __init__(self, texts_tensor, labels_tensor):
        self.texts = texts_tensor
        self.labels = labels_tensor

    def __len__(self):
        return self.texts.size(0)

    def __getitem__(self, idx):
        input_ids = self.texts[idx]
        attention_masks = (input_ids != vocab_to_int["<pad>"]).long()
        segment_ids = torch.zeros_like(input_ids)
        labels = self.labels[idx]
        return input_ids, attention_masks, segment_ids, labels


# numpy X/labels -> torch
X_torch = torch.tensor(X, dtype=torch.long)
y_torch = torch.tensor(labels.squeeze(1), dtype=torch.long)  # (N,) with 0/1

# train/dev/test
train_texts, train_labels = X_torch[:4], y_torch[:4]
dev_texts, dev_labels     = X_torch[4:5], y_torch[4:5]
test_texts, test_labels   = X_torch[5:],  y_torch[5:]

In [ ]:
# Data Preparation & Create a Model Instance using Pytorch

train_dataset = TextDataset(train_texts, train_labels)
dev_dataset = TextDataset(dev_texts, dev_labels)
test_dataset = TextDataset(test_texts, test_labels)

train_loader = DataLoader(train_dataset, 8, shuffle=True)
dev_loader = DataLoader(dev_dataset, 8, shuffle=True)
test_loader = DataLoader(test_dataset, 8, shuffle=True)

# Define a loss
criterion = nn.CrossEntropyLoss()

# Example usage
input_size = 10  # Number of input features per time step
hidden_size = 20  # Number of features in the hidden state
num_layers = 2  # Number of RNN layers (stacked)
output_size = 2  # Number of output classes

# Create RNN model
model = RNNModel(input_size, hidden_size, num_layers, output_size)

In [ ]:
# Train & Test API using Pytorch (Example)
max_grad_norm = 1.0

# Train API
def train(loader):
    model.train()
    optimizer = SGD(model.parameters(), lr=0.1)

    train_loss = 0.0
    for data in loader:
        input_ids, attention_masks, segment_ids, labels = data

        optimizer.zero_grad()  # (실행 위해 최소 보완: gradient 누적 방지)

        out = model(input_ids)
        loss = criterion(out, labels)
        loss.backward()
        train_loss += loss.item()
        if max_grad_norm > 0.:
            nn.utils.clip_grad_norm_(model.parameters(),max_grad_norm)

        optimizer.step()
    return train_loss/len(train_loader)

# Test API
def evaluate(loader):
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    y_true, y_pred = [], []
    loader = tqdm(loader, leave=False)

    for i, data in enumerate(loader):
        with torch.no_grad():
            input_ids, attention_masks, segment_ids, labels = data
            output = model(input_ids)
            for j in range(output.size(0)):
                y_pred.append(output[j].argmax().item())
                y_true.append(labels[j].item())

            loss = criterion(output,labels)
            eval_loss += loss.item()
            loader.set_description(f'eval loss = {(eval_loss / (i+1)):.6f}')
    eval_acc = accuracy_score(y_true, y_pred) * 100.
    return eval_loss / len(loader), eval_acc

for epoch in range(100):
    tr_loss = train(train_loader)
    te_loss, te_acc = evaluate(test_loader)
    print(f"[PyTorch RNN] epoch={epoch:02d} train_loss={tr_loss:.4f} | test_loss={te_loss:.4f} test_acc={te_acc:.1f}%")

  0%|          | 0/1 [00:00<?, ?it/s]

[PyTorch RNN] epoch=00 train_loss=0.5987 | test_loss=0.8547 test_acc=0.0%


  0%|          | 0/1 [00:00<?, ?it/s]

[PyTorch RNN] epoch=01 train_loss=0.5570 | test_loss=0.8862 test_acc=0.0%


  0%|          | 0/1 [00:00<?, ?it/s]

...

[PyTorch RNN] epoch=97 train_loss=0.0063 | test_loss=1.0427 test_acc=0.0%


  0%|          | 0/1 [00:00<?, ?it/s]

[PyTorch RNN] epoch=98 train_loss=0.0062 | test_loss=1.0415 test_acc=0.0%


  0%|          | 0/1 [00:00<?, ?it/s]

[PyTorch RNN] epoch=99 train_loss=0.0061 | test_loss=1.0402 test_acc=0.0%
